## Entity Matching Notebook for Stage 3 of Data Science Project
### Game Website Dataset from Gamespot and howLongToBeat

In [33]:
import py_entitymatching as em
import pandas as pd
import sys
import math
from datetime import datetime

# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
pandas version: 0.20.1
magellan version: 0.2.0


#### Reading Input dataset

In [3]:
#A - Gamespot.csv
#B - howLongToBeat.csv

dfTableA = em.read_csv_metadata('../data/Gamespot.csv', key='ID')
dfTableB = em.read_csv_metadata('../data/howLongToBeat.csv', key='ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
print('Number of tuples in A: ' + str(len(dfTableA)))
print('Number of tuples in B: ' + str(len(dfTableB)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(dfTableA)*len(dfTableB)))

Number of tuples in A: 8884
Number of tuples in B: 8481
Number of tuples in A X B (i.e the cartesian product): 75345204


In [5]:
dfTableA.head()

,ID,Title,Developer,Publisher,Platform,Genre,ReleaseDate,Rating,CleanRDate
0,a1,.hack//G.U. vol. 1//Rebirth,CyberConnect2,Namco Bandai Games|Bandai,PlayStation 2,Action|Role-Playing,24 Oct 2006,8.8,2006-10-24
1,a2,.hack//G.U. vol. 2//Reminisce,CyberConnect2,Bandai|Namco Bandai Games,PlayStation 2,Action|Role-Playing,8 May 2007,8.9,2007-05-08
2,a3,.hack//Infection Part 1,CyberConnect2,Bandai,PlayStation 2,Action|Role-Playing,10 Feb 2003,7.9,2003-02-10
3,a4,.hack//Mutation Part 2,CyberConnect2,Bandai,PlayStation 2,Role-Playing|Action,6 May 2003,7.9,2003-05-06
4,a5,.hack//Outbreak Part 3,CyberConnect2,Bandai,PlayStation 2,Action|Role-Playing,9 Sep 2003,8.0,2003-09-09


In [6]:
dfTableB.head()

,ID,Title,Developer,Publisher,Platform,Genre,ReleaseDate,Rating,CleanRDate
0,b1,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,16 October 2012,5.0,2012-10-16
1,b2,007: The World Is Not Enough,Eurocom|Black Ops,Electronic Arts,Nintendo 64|PlayStation,First-Person|Shooter,17 October 2000,7.0,2000-10-17
2,b3,0RBITALIS,Alan Zucconi,NaN,PC,Puzzle,16 April 2014,8.0,2014-04-16
3,b4,10000000,EightyEight Games,EightyEight Games,Android|iOS|Linux|PC,Role-Playing,29 August 2012,7.7,2012-08-29
4,b5,1000 Amps,Brandon Brizzi,Brandon Brizzi,PC,Adventure|Platform|Puzzle,22 February 2012,6.9,2012-02-22


### Blocking Table to get Candidate Set

#### Try Overlap Blocker
Starting with q-gram overlap

In [7]:
ob = em.OverlapBlocker()

In [17]:
C1 = ob.block_tables(dfTableA, dfTableB, 'Title', 'Title', q_val=3, word_level=False, overlap_size=4, rem_stop_words=True,
                    l_output_attrs=['Title', 'Developer', 'Publisher','Platform','Genre','Rating','ReleaseDate', 'CleanRDate'],
                   r_output_attrs=['Title', 'Developer', 'Publisher', 'Platform','Genre','Rating','ReleaseDate', 'CleanRDate'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


In [18]:
len(C1)

979045

In [19]:
C1.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_Rating,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_Rating,rtable_ReleaseDate,rtable_CleanRDate
0,0,a8,b1,007 Legends,Eurocom Entertainment Software,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter|3D|Action,5.1,16 Oct 2012,2012-10-16,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
1,1,a10,b1,007: The World is not Enough,Eurocom Entertainment Software|Black Ops Entertainment,Electronic Arts,Nintendo 64|PlayStation,Action|First-Person|Shooter|3D,7.6,1 Nov 2000,2000-11-01,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
2,2,a11,b1,007: Tomorrow Never Dies,Black Ops Entertainment,MGM Interactive|EA Games|Electronic Arts|Electronic Arts Victor,PlayStation,Third-Person|3D|Action|Shooter,6.3,16 Nov 1999,1999-11-16,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
3,3,a2954,b1,GoldenEye 007 (2010),n-Space|Eurocom Entertainment Software,Activision|Nintendo,DS|Wii,Shooter|3D|Action|First-Person,8.4,2 Nov 2010,2010-11-02,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
4,4,a2955,b1,GoldenEye 007: Reloaded,Eurocom Entertainment Software,Activision,PlayStation 3|Xbox 360,First-Person|Shooter|3D|Action,7.3,1 Nov 2011,2011-11-01,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16


### Debug Overlap blocker output

In [20]:
dbg1 = em.debug_blocker(C1, dfTableA, dfTableB)

In [21]:
dbg1

,_id,similarity,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_ReleaseDate,rtable_CleanRDate
0,0,1.000000,a1492,b1479,D,WARP,Panasonic|WARP|Sanyei Shobou|Night Dive Studios|Acclaim|Acclaim Japan|Tec Toy,3DO|Linux|Macintosh|PC|PlayStation|Saturn,Adventure,1995,1995-01-01,D,WARP,Acclaim Entertainment,PC|PlayStation|Sega Saturn,First-Person|Adventure|Puzzle|Survival|Survival Horror,1995,1995-01-01
1,1,0.625000,a4804,b397,Naval Warfare,Games Distillery,Just A Game|Microsoft Game Studios,PC|Xbox 360,Action|Fixed-Screen|Shooter|2D,19 May 2010,2010-05-19,Aqua,Games Distillery,Microsoft Studios,PC|Xbox 360,Shooter,19 May 2010,2010-05-19
2,2,0.625000,a6844,b5805,SSX,EA Canada,Electronic Arts|EA Sports,PlayStation 3|Xbox 360,Snowboarding/Skiing|Sports,28 Feb 2012,2012-02-28,SSX (2012),EA Canada,EA Sports,PlayStation 3|Xbox 360,Sports,28 February 2012,2012-02-28
3,3,0.545455,a4923,b42,NCAA GameBreaker,Sony Interactive Studios America,SCEA,PlayStation,Sports|Team-Based|Football (American)|Simulation,31 Aug 1996,1996-08-31,2Xtreme,Sony Interactive Studios America,SCEA,PlayStation,Racing|Sports,31 October 1996,1996-10-31
4,4,0.500000,a8641,b8252,Worms 2,Team 17,MicroProse|Hasbro Interactive,PC,Strategy|Turn-Based,31 Dec 1998,1998-12-31,Worms Pinball,Team 17,Team 17,PC,Arcade|Pinball,1998,1998-01-01
5,5,0.500000,a7710,b7501,The Swapper,Facepalm Games,Facepalm Games|Curve Studios|Curve Digital Games|Limited Run Games|Nintendo,Linux|Macintosh|PC|PlayStation 3|PlayStation 4|PlayStation Vita|Wii U|Xbox One,2D|Action|Platformer,30 May 2013,2013-05-30,The Yawhg,NaN,NaN,PC,Adventure,30 May 2013,2013-05-30
6,6,0.500000,a8639,b1241,Worldwide Soccer,Sega,Sega|Expert Software|Empire Interactive|Tec Toy|Samsung,PC|Saturn,Soccer|Sports|Team-Based|Simulation,11 May 1995,1995-05-11,Clockwork Knight,Sega,Sega,Sega Saturn,Scrolling|Platform,11 May 1995,1995-05-11
7,7,0.500000,a1821,b1269,DiRT,Codemasters,Mastertronic|Codemasters|Sold Out Software,PC|PlayStation 3|Xbox 360,Driving/Racing|Simulation,19 Jun 2007,2007-06-19,Colin McRae: Dirt,Codemasters,Codemasters,PC|PlayStation 3|Xbox 360,Racing,19 June 2007,2007-06-19
8,8,0.461538,a7403,b6938,The Ant Bully,Artificial Mind and Movement,Midway,Game Boy Advance|GameCube|PC|PlayStation 2|Wii,Action|Platformer|3D,24 Jul 2006,2006-07-24,Teen Titans,Artificial Mind and Movement,THQ,Nintendo GameCube|PlayStation 2,Action,24 May 2006,2006-05-24
9,9,0.454545,a305,b5905,Anomaly 2,11 bit studios,11 bit studios|Chillingo Ltd,Android|BlackBerry|iOS (iPhone/iPad)|Linux|Macintosh|PC|PlayStation 4,Real-Time|Strategy,15 May 2013,2013-05-15,Sanctum 2,Coffee Stain Studios,NaN,PC,First-Person|Shooter|Tower Defense,15 May 2013,2013-05-15


#### The number of candidates upon using overlapping blocker has reduced from 75345204 to 979045. Debugger is not showing that there are several potential matches dropped. Using overlap blocker again on candidate set. Blocking by entire keyword.

In [22]:
C2 = ob.block_candset(C1, 'Title', 'Title', word_level=True, overlap_size=1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


In [23]:
len(C2)

530183

In [24]:
C2.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_Rating,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_Rating,rtable_ReleaseDate,rtable_CleanRDate
0,0,a8,b1,007 Legends,Eurocom Entertainment Software,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter|3D|Action,5.1,16 Oct 2012,2012-10-16,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
1,1,a10,b1,007: The World is not Enough,Eurocom Entertainment Software|Black Ops Entertainment,Electronic Arts,Nintendo 64|PlayStation,Action|First-Person|Shooter|3D,7.6,1 Nov 2000,2000-11-01,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
2,2,a11,b1,007: Tomorrow Never Dies,Black Ops Entertainment,MGM Interactive|EA Games|Electronic Arts|Electronic Arts Victor,PlayStation,Third-Person|3D|Action|Shooter,6.3,16 Nov 1999,1999-11-16,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
3,3,a2954,b1,GoldenEye 007 (2010),n-Space|Eurocom Entertainment Software,Activision|Nintendo,DS|Wii,Shooter|3D|Action|First-Person,8.4,2 Nov 2010,2010-11-02,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
4,4,a2955,b1,GoldenEye 007: Reloaded,Eurocom Entertainment Software,Activision,PlayStation 3|Xbox 360,First-Person|Shooter|3D|Action,7.3,1 Nov 2011,2011-11-01,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16


In [25]:
dbg1_2 = em.debug_blocker(C2, dfTableA, dfTableB)

In [26]:
dbg1_2

,_id,similarity,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_ReleaseDate,rtable_CleanRDate
0,0,1.000000,a1492,b1479,D,WARP,Panasonic|WARP|Sanyei Shobou|Night Dive Studios|Acclaim|Acclaim Japan|Tec Toy,3DO|Linux|Macintosh|PC|PlayStation|Saturn,Adventure,1995,1995-01-01,D,WARP,Acclaim Entertainment,PC|PlayStation|Sega Saturn,First-Person|Adventure|Puzzle|Survival|Survival Horror,1995,1995-01-01
1,1,0.625000,a6844,b5805,SSX,EA Canada,Electronic Arts|EA Sports,PlayStation 3|Xbox 360,Snowboarding/Skiing|Sports,28 Feb 2012,2012-02-28,SSX (2012),EA Canada,EA Sports,PlayStation 3|Xbox 360,Sports,28 February 2012,2012-02-28
2,2,0.625000,a4804,b397,Naval Warfare,Games Distillery,Just A Game|Microsoft Game Studios,PC|Xbox 360,Action|Fixed-Screen|Shooter|2D,19 May 2010,2010-05-19,Aqua,Games Distillery,Microsoft Studios,PC|Xbox 360,Shooter,19 May 2010,2010-05-19
3,3,0.555556,a109,b170,ADR1FT,Three One Zero,505 Games,PC|PlayStation 4|Xbox One,Action|VR|Adventure|3D|Survival,28 Mar 2016,2016-03-28,Adrift,Three One Zero,505 Games,PC|PlayStation 4,Adventure,28 March 2016,2016-03-28
4,4,0.545455,a4923,b42,NCAA GameBreaker,Sony Interactive Studios America,SCEA,PlayStation,Sports|Team-Based|Football (American)|Simulation,31 Aug 1996,1996-08-31,2Xtreme,Sony Interactive Studios America,SCEA,PlayStation,Racing|Sports,31 October 1996,1996-10-31
5,5,0.545455,a29,b42,2 Xtreme,Sony Interactive Studios America,SCEA|SCEE|SCEI,PlayStation,Arcade|Driving/Racing,31 Oct 1996,1996-10-31,2Xtreme,Sony Interactive Studios America,SCEA,PlayStation,Racing|Sports,31 October 1996,1996-10-31
6,6,0.500000,a7710,b7501,The Swapper,Facepalm Games,Facepalm Games|Curve Studios|Curve Digital Games|Limited Run Games|Nintendo,Linux|Macintosh|PC|PlayStation 3|PlayStation 4|PlayStation Vita|Wii U|Xbox One,2D|Action|Platformer,30 May 2013,2013-05-30,The Yawhg,NaN,NaN,PC,Adventure,30 May 2013,2013-05-30
7,7,0.500000,a8639,b1241,Worldwide Soccer,Sega,Sega|Expert Software|Empire Interactive|Tec Toy|Samsung,PC|Saturn,Soccer|Sports|Team-Based|Simulation,11 May 1995,1995-05-11,Clockwork Knight,Sega,Sega,Sega Saturn,Scrolling|Platform,11 May 1995,1995-05-11
8,8,0.500000,a8641,b8252,Worms 2,Team 17,MicroProse|Hasbro Interactive,PC,Strategy|Turn-Based,31 Dec 1998,1998-12-31,Worms Pinball,Team 17,Team 17,PC,Arcade|Pinball,1998,1998-01-01
9,9,0.500000,a1821,b1269,DiRT,Codemasters,Mastertronic|Codemasters|Sold Out Software,PC|PlayStation 3|Xbox 360,Driving/Racing|Simulation,19 Jun 2007,2007-06-19,Colin McRae: Dirt,Codemasters,Codemasters,PC|PlayStation 3|Xbox 360,Racing,19 June 2007,2007-06-19


### Trying Attribute Equivalence Blocker

In [27]:
attrB = em.AttrEquivalenceBlocker()

In [28]:
C3 = attrB.block_tables(dfTableA, dfTableB, l_block_attr='Title', r_block_attr='Title',
                       l_output_attrs=['Title', 'Developer', 'Publisher','Platform','Genre','Rating','ReleaseDate', 'CleanRDate'],
                   r_output_attrs=['Title', 'Developer', 'Publisher', 'Platform','Genre','Rating','ReleaseDate', 'CleanRDate'])

In [29]:
len(C3)

3480

In [30]:
C3.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_Rating,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_Rating,rtable_ReleaseDate,rtable_CleanRDate
0,0,a8,b1,007 Legends,Eurocom Entertainment Software,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter|3D|Action,5.1,16 Oct 2012,2012-10-16,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
1,1,a12,b12,1-2-Switch,Nintendo,Nintendo,Nintendo Switch,Party/Minigame,2.0,3 Mar 2017,2017-03-03,1-2-Switch,Nintendo EPD,Nintendo,Nintendo Switch,Action|Party,5.4,03 March 2017,2017-03-03
2,2,a18,b19,140,Jeppe Carlsen,Jeppe Carlsen|Double Fine Productions|Abstraction Games,Linux|Macintosh|PC|PlayStation 4|PlayStation Vita|Wii U|Xbox One,Action|Platformer|2D,7.6,16 Oct 2013,2013-10-16,140,Jeppe Carlsen,NaN,Linux|Mac|PC,Music/Rhythm|Platform,7.4,16 October 2013,2013-10-16
3,3,a25,b24,1942: Joint Strike,Backbone Entertainment,Capcom,PlayStation 3|Xbox 360,Shoot-'Em-Up|2D|Action,7.2,23 Jul 2008,2008-07-23,1942: Joint Strike,Backbone Entertainment,Capcom,PlayStation 3|Xbox 360,Vertical scrolling shooter,6.6,23 July 2008,2008-07-23
4,4,a26,b23,1942,Elite Systems Ltd.|Capcom|Digital Eclipse|Micronics|Syrox Developments,Elite Systems Ltd.|Capcom|Elite|ASCII Entertainment|Zafiro|MCM,Amstrad CPC|Android|Arcade Games|Commodore 64|FM-7|Game Boy Color|iOS (iPhone/iPad)|Mobile|MSX|N...,2D|Action|Shoot-'Em-Up,7.6,1984,1984-01-01,1942,Capcom,Capcom,Arcade|NES,NaN,6.2,December 1984,1984-12-01


In [31]:
dbg2 = em.debug_blocker(C3, dfTableA, dfTableB)

In [32]:
dbg2

,_id,similarity,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_ReleaseDate,rtable_CleanRDate
0,0,1.000000,a5169,b4890,Not a Hero,roll7,Devolver Digital|Team 17,Linux|Macintosh|PC|PlayStation 4|Xbox One,Action|Shoot-'Em-Up|2D,14 May 2015,2015-05-14,Not A Hero,Roll7,Devolver Digital,PC|PlayStation 4,Shooter,14 May 2015,2015-05-14
1,1,0.888889,a5395,b5116,Perception,The Deep End Games,The Deep End Games|WFH GAMES SP. Z O.O.|Feardemic,Nintendo Switch|PC|PlayStation 4|Xbox One,First-Person|Adventure|3D,30 May 2017,2017-05-30,Perception (2017),The Deep End Games,Feardemic,PC,First-Person|Adventure|Survival Horror,30 May 2017,2017-05-30
2,2,0.875000,a2831,b2810,Game of Thrones,Cyanide,ATLUS|Focus Home Interactive,PC|PlayStation 3|Xbox 360,Role-Playing,15 May 2012,2012-05-15,Game of Thrones (2012),Cyanide,Atlus|Focus Home Interactive,PC|PlayStation 3|Xbox 360,Role-Playing,15 May 2012,2012-05-15
3,3,0.857143,a3442,b8469,inFamous,Sucker Punch,SCEI|SCE Australia|SCEA|SCEE,PlayStation 3,Action|Adventure,26 May 2009,2009-05-26,inFAMOUS,Sucker Punch Productions,Sony Computer Entertainment,PlayStation 3,Action|Adventure|Open World,26 May 2009,2009-05-26
4,4,0.846154,a5692,b5369,Psychonauts in The Rhombus of Ruin,Double Fine Productions,Sony Interactive Entertainment|Double Fine Productions,PlayStation 4,Action|VR,21 Feb 2017,2017-02-21,Psychonauts in the Rhombus of Ruin,Double Fine Productions,Double Fine Productions,PlayStation 4|PlayStation VR,Virtual Reality|Adventure,21 February 2017,2017-02-21
5,5,0.833333,a6951,b6489,Starcraft II: Heart of the Swarm,Blizzard Entertainment,Activision|Blizzard Entertainment,Macintosh|PC,Strategy|Real-Time,12 Mar 2013,2013-03-12,StarCraft II: Heart of the Swarm,Blizzard Entertainment,Blizzard Entertainment,Mac|PC,Top-Down|Real-Time|Strategy,12 March 2013,2013-03-12
6,6,0.833333,a7265,b6884,Tales From Space: Mutant Blobs Attack,DrinkBox Studios,DrinkBox Studios|Midnight City,Linux|Macintosh|PC|PlayStation 3|PlayStation Vita|Xbox 360,2D|Action|Platformer,21 Feb 2012,2012-02-21,Tales from Space: Mutant Blobs Attack,Drinkbox Studios,Drinkbox Studios,Mac|PC|PlayStation 3|PlayStation Vita|Xbox 360,Side|Scrolling|Platform,21 February 2012,2012-02-21
7,7,0.833333,a8852,b8410,ZENONIA,GAMEVIL Inc.,GAMEVIL Inc.,Android|DS|iOS (iPhone/iPad)|PSP|Windows Mobile|Zeebo,Action|Role-Playing,24 May 2009,2009-05-24,Zenonia,Gamevil,Gamevil,Android|iOS|Mobile|Nintendo 3DS|Nintendo DS,Action RPG,24 May 2009,2009-05-24
8,8,0.818182,a4416,b4432,Middle-earth: Shadow of Mordor,Monolith Productions|Behaviour Interactive,Warner Bros. Interactive Entertainment,Linux|Macintosh|PC|PlayStation 3|PlayStation 4|Xbox 360|Xbox One,Action|Adventure,30 Sep 2014,2014-09-30,Middle-Earth: Shadow of Mordor,Monolith Productions|Behaviour Interactive,Warner Bros. Interactive Entertainment,Linux|Mac|PC|PlayStation 3|PlayStation 4|Xbox 360|Xbox One,Action|Open World|Role-Playing,30 September 2014,2014-09-30
9,9,0.818182,a6952,b6490,Starcraft II: Legacy of the Void,Blizzard Entertainment,Blizzard Entertainment,Macintosh|PC,Strategy|Real-Time,10 Nov 2015,2015-11-10,StarCraft II: Legacy of the Void,Blizzard Entertainment,Blizzard Entertainment,Mac|PC,Real-Time|Strategy,2015,2015-01-01


#### Attribute equivalence blocker generates candidate set with almost all positives but since it is case sensitive we are missing on lot of positive samples.

#### Trying Rule based blocking on Candidate set C2 generated after running Overlap blocker by q-gram and word overlap

In [38]:
block_f = em.get_features_for_blocking(dfTableA, dfTableB, validate_inferred_attr_types=False)

In [41]:
block_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,ID_ID_lev_dist,ID,ID,None,None,lev_dist,<function ID_ID_lev_dist at 0x000002025D589158>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,ID_ID_lev_sim,ID,ID,None,None,lev_sim,<function ID_ID_lev_sim at 0x000002020181BE18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,ID_ID_jar,ID,ID,None,None,jaro,<function ID_ID_jar at 0x000002020181B730>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,ID_ID_jwn,ID,ID,None,None,jaro_winkler,<function ID_ID_jwn at 0x000002020181BB70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x000002020181B950>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,ID_ID_jac_qgm_3_qgm_3,ID,ID,qgm_3,qgm_3,jaccard,<function ID_ID_jac_qgm_3_qgm_3 at 0x000002020181BC80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Title_Title_jac_qgm_3_qgm_3,Title,Title,qgm_3,qgm_3,jaccard,<function Title_Title_jac_qgm_3_qgm_3 at 0x000002020181BAE8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Title_Title_cos_dlm_dc0_dlm_dc0,Title,Title,dlm_dc0,dlm_dc0,cosine,<function Title_Title_cos_dlm_dc0_dlm_dc0 at 0x000002020181B598>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Title_Title_jac_dlm_dc0_dlm_dc0,Title,Title,dlm_dc0,dlm_dc0,jaccard,<function Title_Title_jac_dlm_dc0_dlm_dc0 at 0x000002020181BBF8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Title_Title_mel,Title,Title,None,None,monge_elkan,<function Title_Title_mel at 0x00000202077750D0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


#### Add similarity measure as rule for candidate in set C2

In [175]:
rb = em.RuleBasedBlocker()
rb.add_rule(['Title_Title_lev_sim(ltuple, rtuple) < 0.4'], block_f)

'_rule_0'

In [176]:
C4 = rb.block_candset(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:14


In [177]:
len(C4)

70668

In [178]:
C4.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_Rating,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_Rating,rtable_ReleaseDate,rtable_CleanRDate
0,0,a8,b1,007 Legends,Eurocom Entertainment Software,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter|3D|Action,5.1,16 Oct 2012,2012-10-16,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
8,8,a973,b1,Burnout Legends,Visual Impact|Criterion Games,EA Games|Electronic Arts,DS|PSP,Arcade|Driving/Racing,8.7,13 Sep 2005,2005-09-13,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
14,14,a2557,b1,Fighting Legends,Maximum Charisma,Maximum Charisma,PC,Role-Playing,6.6,31 Oct 2001,2001-10-31,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
15,15,a3826,b1,League of Legends,Riot Games,TransGaming Inc.|Riot Games|Garena,Macintosh|PC,Free-to-Play|MOBA|Strategy,8.0,27 Oct 2009,2009-10-27,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16
24,24,a4295,b1,Mega Man Legends 2,Capcom,Capcom,PC|PlayStation|PSP,Action|Role-Playing,8.7,24 Oct 2000,2000-10-24,007 Legends,Eurocom,Activision,PC|PlayStation 3|Wii U|Xbox 360,First-Person|Shooter,5.0,16 October 2012,2012-10-16


In [179]:
dbg1_3 = em.debug_blocker(C4, dfTableA, dfTableB)

In [180]:
dbg1_3

,_id,similarity,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_ReleaseDate,rtable_CleanRDate
0,0,1.000000,a1492,b1479,D,WARP,Panasonic|WARP|Sanyei Shobou|Night Dive Studios|Acclaim|Acclaim Japan|Tec Toy,3DO|Linux|Macintosh|PC|PlayStation|Saturn,Adventure,1995,1995-01-01,D,WARP,Acclaim Entertainment,PC|PlayStation|Sega Saturn,First-Person|Adventure|Puzzle|Survival|Survival Horror,1995,1995-01-01
1,1,0.857143,a3442,b8469,inFamous,Sucker Punch,SCEI|SCE Australia|SCEA|SCEE,PlayStation 3,Action|Adventure,26 May 2009,2009-05-26,inFAMOUS,Sucker Punch Productions,Sony Computer Entertainment,PlayStation 3,Action|Adventure|Open World,26 May 2009,2009-05-26
2,2,0.833333,a8852,b8410,ZENONIA,GAMEVIL Inc.,GAMEVIL Inc.,Android|DS|iOS (iPhone/iPad)|PSP|Windows Mobile|Zeebo,Action|Role-Playing,24 May 2009,2009-05-24,Zenonia,Gamevil,Gamevil,Android|iOS|Mobile|Nintendo 3DS|Nintendo DS,Action RPG,24 May 2009,2009-05-24
3,3,0.714286,a2201,b2185,Elex,Piranha Bytes,THQ Nordic,PC|PlayStation 4|Xbox One,Role-Playing,17 Oct 2017,2017-10-17,ELEX,Piranha Bytes,THQ Nordic,PC|PlayStation 4,Open World|Role-Playing,17 October 2017,2017-10-17
4,4,0.714286,a6134,b5462,Ruiner,Reikon Games,Devolver Digital,Linux|Macintosh|PC|PlayStation 4|Xbox One,Action,26 Sep 2017,2017-09-26,RUINER,Reikon Games,Devolver Digital,PC|PlayStation 4|Xbox One,Action,26 September 2017,2017-09-26
5,5,0.714286,a6218,b5909,SATAZIUS,Astro Port,Nyu Media|Capcom,Linux|PC,Shoot-'Em-Up|2D|Action,16 Dec 2011,2011-12-16,Satazius,ASTRO PORT,Capcom,PC,Action,16 December 2011,2011-12-16
6,6,0.666667,a3902,b3913,LIMBO,PLAYDEAD,PLAYDEAD|Merge Games|Microsoft Game Studios,Android|iOS (iPhone/iPad)|Linux|Macintosh|PC|PlayStation 3|PlayStation 4|PlayStation Vita|Xbox 3...,Action|Platformer|2D,21 Jul 2010,2010-07-21,Limbo,Playdead,Playdead|Microsoft Game Studios,Android|iOS|Linux|Mac|PC|PlayStation 3|PlayStation 4|PlayStation Vita|Xbox 360|Xbox One,Side|Scrolling|Platform|Puzzle,21 July 2010,2010-07-21
7,7,0.666667,a2491,b2497,FATE,WildTangent,WildTangent|Encore Software|Inc.,Macintosh|PC,Action|Role-Playing,18 May 2005,2005-05-18,Fate,WildStudios|WildTangent,WildTangent|Encore Software,PC,Action|Role-Playing,18 May 2005,2005-05-18
8,8,0.666667,a3460,b3463,INSIDE,PLAYDEAD,PLAYDEAD,Macintosh|PC|PlayStation 4|Xbox One,Platformer|2D|Action,29 Jun 2016,2016-06-29,Inside,PlayDead,PlayDead,PC|PlayStation 4|Xbox One,NaN,29 June 2016,2016-06-29
9,9,0.625000,a2334,b2329,EVERYTHING,David OReilly,David OReilly,Linux|Macintosh|PC|PlayStation 4,Adventure,21 Mar 2017,2017-03-21,Everything (2017),DAVID OREILLY,DAVID OREILLY,PC|PlayStation 4,Simulation,21 March 2017,2017-03-21


#### A few single word titles are blocked mainly because the blocking is case-sensitive to title attribute. A case-insensitive version will help reduce the number of dropped matches.

Reducing the candidate blockset further by using black box matcher with custom function to check for release date similarity.

In [110]:
# release date compare function
# if titles release match within a year in both tables we don't block it
def compareReleaseDate(A, B):
    A_cleanRDate = A['CleanRDate']
    B_cleanRDate = B['CleanRDate']
    
    try:
        d1 = datetime.strptime(A_cleanRDate, "%Y-%m-%d")
        d2 = datetime.strptime(B_cleanRDate, "%Y-%m-%d")
        dayDiff = abs(d2-d1).days

        if dayDiff > 366:
            return True
    except:
        return False

In [111]:
bb = em.BlackBoxBlocker()
# Set the black box function
bb.set_black_box_function(compareReleaseDate)
C5 = bb.block_candset(C4)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


In [112]:
len(C5)

13062

### Combine candidates

In [181]:
# combining true matches as found by Attribute equivalence blocker
C = em.combine_blocker_outputs_via_union([C3, C5])
len(C)

13186

In [205]:
# save candidate set C to file
C.to_csv('selectedCandidates.csv',index=False)

In [167]:
dbg1_4 = em.debug_blocker(C, dfTableA, dfTableB)

In [169]:
dbg1_4

,_id,similarity,ltable_ID,rtable_ID,ltable_Title,ltable_Developer,ltable_Publisher,ltable_Platform,ltable_Genre,ltable_ReleaseDate,ltable_CleanRDate,rtable_Title,rtable_Developer,rtable_Publisher,rtable_Platform,rtable_Genre,rtable_ReleaseDate,rtable_CleanRDate
0,0,0.857143,a3442,b8469,inFamous,Sucker Punch,SCEI|SCE Australia|SCEA|SCEE,PlayStation 3,Action|Adventure,26 May 2009,2009-05-26,inFAMOUS,Sucker Punch Productions,Sony Computer Entertainment,PlayStation 3,Action|Adventure|Open World,26 May 2009,2009-05-26
1,1,0.833333,a8852,b8410,ZENONIA,GAMEVIL Inc.,GAMEVIL Inc.,Android|DS|iOS (iPhone/iPad)|PSP|Windows Mobile|Zeebo,Action|Role-Playing,24 May 2009,2009-05-24,Zenonia,Gamevil,Gamevil,Android|iOS|Mobile|Nintendo 3DS|Nintendo DS,Action RPG,24 May 2009,2009-05-24
2,2,0.750000,a7512,b7171,The Incredible Adventures of Van Helsing III,NeocoreGames,NeocoreGames,Macintosh|PC|Xbox One,Action|Role-Playing,22 May 2015,2015-05-22,The Incredible Adventures of Van Helsing,NeocoreGames,NeocoreGames,PC|Xbox One,Action|Role-Playing,22 May 2013,2013-05-22
3,3,0.714286,a6218,b5909,SATAZIUS,Astro Port,Nyu Media|Capcom,Linux|PC,Shoot-'Em-Up|2D|Action,16 Dec 2011,2011-12-16,Satazius,ASTRO PORT,Capcom,PC,Action,16 December 2011,2011-12-16
4,4,0.714286,a6134,b5462,Ruiner,Reikon Games,Devolver Digital,Linux|Macintosh|PC|PlayStation 4|Xbox One,Action,26 Sep 2017,2017-09-26,RUINER,Reikon Games,Devolver Digital,PC|PlayStation 4|Xbox One,Action,26 September 2017,2017-09-26
5,5,0.714286,a2201,b2185,Elex,Piranha Bytes,THQ Nordic,PC|PlayStation 4|Xbox One,Role-Playing,17 Oct 2017,2017-10-17,ELEX,Piranha Bytes,THQ Nordic,PC|PlayStation 4,Open World|Role-Playing,17 October 2017,2017-10-17
6,6,0.666667,a2781,b2763,Fullmetal Alchemist and the Broken Angel,Racjin,Square Enix,PlayStation 2,Role-Playing|Action,18 Jan 2005,2005-01-18,FullMetal Alchemist And The Broken Angel,Racjin,Square Enix,PlayStation 2,Action role-playing,18 January 2003,2003-01-18
7,7,0.666667,a2491,b2497,FATE,WildTangent,WildTangent|Encore Software|Inc.,Macintosh|PC,Action|Role-Playing,18 May 2005,2005-05-18,Fate,WildStudios|WildTangent,WildTangent|Encore Software,PC,Action|Role-Playing,18 May 2005,2005-05-18
8,8,0.666667,a3902,b3913,LIMBO,PLAYDEAD,PLAYDEAD|Merge Games|Microsoft Game Studios,Android|iOS (iPhone/iPad)|Linux|Macintosh|PC|PlayStation 3|PlayStation 4|PlayStation Vita|Xbox 3...,Action|Platformer|2D,21 Jul 2010,2010-07-21,Limbo,Playdead,Playdead|Microsoft Game Studios,Android|iOS|Linux|Mac|PC|PlayStation 3|PlayStation 4|PlayStation Vita|Xbox 360|Xbox One,Side|Scrolling|Platform|Puzzle,21 July 2010,2010-07-21
9,9,0.666667,a3460,b3463,INSIDE,PLAYDEAD,PLAYDEAD,Macintosh|PC|PlayStation 4|Xbox One,Platformer|2D|Action,29 Jun 2016,2016-06-29,Inside,PlayDead,PlayDead,PC|PlayStation 4|Xbox One,NaN,29 June 2016,2016-06-29


#### Candidate set is now sufficiently reduced to get sufficient positive samples while labling for matcher to learn.

### Final blocking sequence

In [ ]:
##### A, B ----- overlap blocker (3-gram, overlap=4) 
#                                  -----> overlap blocker (1-word) 
#                                             -----> Rule Based Blocker (Title_Title_Sim < 0.4) 
#                                                                        -----> Black Box Blocker - C5
#                                                                                                       | --> C
#     A, B ------ Attr Eq Blocker (Title) --------------------------------------------------------- C3

### Sampling and Labeling

#### Select 400 tuples for labeling to be used by matcher algorithm

In [183]:
S1 = em.sample_table(C, 400)
G1 = em.label_table(S1,'gold_label')
G1.to_csv('toLabel.csv', index=False)

Column name (gold_label) is not present in dataframe


#### Tuples were labeled offline and read back here for further processing

In [113]:
G = em.read_csv_metadata('../data/Labelled.csv', 
                         key='_id',
                         ltable=dfTableA, rtable=dfTableB, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')
len(G)

Metadata file is not present in the given path; proceeding to read the csv file.


400

### Splitting labeled dataset into development and evaluation set

In [186]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']
print ('Development Set(I) size = ' + str(len(I)))
print ('Evaluation Set(J) size = ' + str(len(J)))

Development Set(I) size = 280
Evaluation Set(J) size = 120


In [206]:
# save set I,J to file
I.to_csv('setI.csv', index=False)
J.to_csv('setJ.csv', index=False)

### Instantiate matcher objects

In [115]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Generate feature set for given data

In [116]:
# Generate features
feature_table = em.get_features_for_matching(dfTableA, dfTableB, validate_inferred_attr_types=False)

In [117]:
# List the names of the features generated
feature_table['feature_name']

0                                  ID_ID_lev_dist
1                                   ID_ID_lev_sim
2                                       ID_ID_jar
3                                       ID_ID_jwn
4                                       ID_ID_exm
5                           ID_ID_jac_qgm_3_qgm_3
6                     Title_Title_jac_qgm_3_qgm_3
7                 Title_Title_cos_dlm_dc0_dlm_dc0
8                 Title_Title_jac_dlm_dc0_dlm_dc0
9                                 Title_Title_mel
10                           Title_Title_lev_dist
11                            Title_Title_lev_sim
12                                Title_Title_nmw
13                                 Title_Title_sw
14            Developer_Developer_jac_qgm_3_qgm_3
15        Developer_Developer_cos_dlm_dc0_dlm_dc0
16        Developer_Developer_jac_dlm_dc0_dlm_dc0
17                        Developer_Developer_mel
18                   Developer_Developer_lev_dist
19                    Developer_Developer_lev_sim


### Convert development set I into feature vectors

In [118]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='gold_label',
                            show_progress=False)

In [119]:
H.head()

,_id,ltable_ID,rtable_ID,ID_ID_lev_dist,ID_ID_lev_sim,ID_ID_jar,ID_ID_jwn,ID_ID_exm,ID_ID_jac_qgm_3_qgm_3,Title_Title_jac_qgm_3_qgm_3,...,Rating_Rating_anm,Rating_Rating_lev_dist,Rating_Rating_lev_sim,CleanRDate_CleanRDate_lev_dist,CleanRDate_CleanRDate_lev_sim,CleanRDate_CleanRDate_jar,CleanRDate_CleanRDate_jwn,CleanRDate_CleanRDate_exm,CleanRDate_CleanRDate_jac_qgm_3_qgm_3,gold_label
92,913082,a7987,b7683,3,0.4,0.600000,0.600000,0,0.0,0.157895,...,0.365854,2.0,0.333333,4.0,0.6,0.8,0.86,0.0,0.142857,0
223,231382,a1983,b2008,5,0.0,0.466667,0.466667,0,0.0,1.000000,...,0.894118,2.0,0.333333,0.0,1.0,1.0,1.00,1.0,1.000000,1
234,304609,a2588,b2606,4,0.2,0.466667,0.466667,0,0.0,0.233333,...,0.692308,2.0,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,0
232,298331,a1418,b2571,5,0.0,0.466667,0.466667,0,0.0,0.361111,...,0.879121,2.0,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,0
377,771435,a843,b6710,5,0.0,0.000000,0.000000,0,0.0,0.307692,...,0.908046,2.0,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,0


### Fill missing feature value computation (NaN) with relevant values

In [120]:
#set all missing value NaN to appropriate numeric values
import math
for idx, row in H.iterrows():
    for column in H.columns:
        v = row[column]
        if isinstance(v, int) or isinstance(v, float):
            if math.isnan(v):
                if column.find('lev_dist') != -1:
                    H.set_value(idx, column, 5)
                else:
                    H.set_value(idx, column, 0)

### Determine best learning algorithm using cross-validation

In [121]:
# Select the best ML matcher using 5-fold CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'],
        k=5,
        target_attr='gold_label', metrics_to_display=['precision', 'recall', 'f1'], random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.965000,0.969318,0.966396
1,RF,1.000000,0.962500,0.979310
2,SVM,0.000000,0.000000,0.000000
3,LinReg,1.000000,0.936667,0.964532
4,LogReg,0.931587,0.944318,0.936896
5,NaiveBayes,0.924373,0.969318,0.944382


### Precision Stats for each fold of CV

In [122]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x0000020217776710>,5,0.9375,0.950000,1.000000,1.000000,0.937500,0.965000
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x00000202177764E0>,5,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x0000020217776390>,5,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x0000020217776470>,5,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x0000020217776F98>,5,1.0000,0.950000,0.888889,0.952381,0.866667,0.931587
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x0000020217776128>,5,0.9375,0.904762,0.842105,1.000000,0.937500,0.924373


### Random Forest get 100% precision at first trial of CV
### Using Random Forest classifier for testing evaluate set J

In [123]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
       target_attr='gold_label')

### Convert evaluation set J into feature vectors and fill missing values

In [124]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold_label', show_progress=False)

#set all missing value NaN to appropriate numeric values
for idx, row in L.iterrows():
    for column in L.columns:
        v = row[column]
        if isinstance(v, int) or isinstance(v, float):
            if math.isnan(v):
                if column.find('lev_dist') != -1:
                    L.set_value(idx, column, 5)
                else:
                    L.set_value(idx, column, 0)

### Predicting the matches

In [125]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

In [190]:
# Evaluate the predictions
print ('Random Forst prediction summary:')
eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Random Forst prediction summary:
Precision : 96.43% (27/28)
Recall : 84.38% (27/32)
F1 : 90.0%
False positives : 1 (out of 28 positive predictions)
False negatives : 5 (out of 92 negative predictions)


In [211]:
JStore = J.copy()
JStore['predictedVal'] = pd.Series()
for idx, row in predictions.iterrows():
    JStore.set_value(idx, 'predictedVal', row['predicted'])
JStore.to_csv('RFprediction.csv', index=False)

# Following section is for completeness of project report

## Evaluating other matchers for Evaluation Set

In [198]:
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
       target_attr='gold_label')

svm.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
       target_attr='gold_label')

lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
       target_attr='gold_label')

ln.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
       target_attr='gold_label')

nb.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
       target_attr='gold_label')

In [199]:
predictionsDT = dt.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

predictionsSVM = svm.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

predictionsLogR = lg.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

predictionsLinR = ln.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

predictionsNB = nb.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

In [200]:
print ('Decision Tree prediction summary:')
eval_result = em.eval_matches(predictionsDT, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Decision Tree prediction summary:
Precision : 90.32% (28/31)
Recall : 87.5% (28/32)
F1 : 88.89%
False positives : 3 (out of 31 positive predictions)
False negatives : 4 (out of 89 negative predictions)


In [201]:
print ('SVM prediction summary:')
eval_result = em.eval_matches(predictionsSVM, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

SVM prediction summary:
Precision : 0.0% (0/0)
Recall : 0.0% (0/32)
F1 : 0.0%
False positives : 0 (out of 0 positive predictions)
False negatives : 32 (out of 120 negative predictions)


In [202]:
print ('Logistic Regression prediction summary:')
eval_result = em.eval_matches(predictionsLogR, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Logistic Regression prediction summary:
Precision : 90.91% (30/33)
Recall : 93.75% (30/32)
F1 : 92.31%
False positives : 3 (out of 33 positive predictions)
False negatives : 2 (out of 87 negative predictions)


In [203]:
print ('Linear Regression prediction summary:')
eval_result = em.eval_matches(predictionsLinR, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Linear Regression prediction summary:
Precision : 93.1% (27/29)
Recall : 84.38% (27/32)
F1 : 88.52%
False positives : 2 (out of 29 positive predictions)
False negatives : 5 (out of 91 negative predictions)


In [204]:
print ('Naive Bayes prediction summary:')
eval_result = em.eval_matches(predictionsNB, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Naive Bayes prediction summary:
Precision : 88.57% (31/35)
Recall : 96.88% (31/32)
F1 : 92.54%
False positives : 4 (out of 35 positive predictions)
False negatives : 1 (out of 85 negative predictions)
